practical 1  :  Implementation of S-DES

In [2]:
import numpy as np

# Permutation and S-box definitions
IP          = (2, 6, 3, 1, 4, 8, 5, 7)
IP_INVERSE  = (4, 1, 3, 5, 7, 2, 8, 6)
E           = (4, 1, 2, 3, 2, 3, 4, 1)
P10         = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8          = (6, 3, 7, 4, 8, 5, 10, 9)
P4          = (2, 4, 3, 1)
S0          = np.array([1,0,3,2,3,2,1,0,0,2,1,3,3,1,3,2]).reshape(4,4)
S1          = np.array([0,1,2,3,2,0,1,3,3,0,1,0,2,1,0,3]).reshape(4,4)

KEY = '0111111101'

# Helper functions
def left(x): return x[:len(x)//2]
def right(x): return x[len(x)//2:]

def permutation(original, key):
    return ''.join(original[i-1] for i in key)

def shift(bit):
    return left(bit)[1:] + left(bit)[0] + right(bit)[1:] + right(bit)[0]

def genfirstKey():
    rotated = shift(permutation(KEY, P10))
    return permutation(rotated, P8)

def gensecondKey():
    rotated = shift(shift(shift(permutation(KEY, P10))))
    return permutation(rotated, P8)

def xor(bit, key):
    return ''.join(str((int(i) + int(j)) % 2) for i, j in zip(bit, key))

def SBox(bit, sbox): 
    row = int(bit[0] + bit[3], 2)
    col = int(bit[1] + bit[2], 2)
    return '{0:02b}'.format(sbox[row][col])

def Fk(bit, key):
    L, R = left(bit), right(bit)
    bit = xor(permutation(R, E), key)
    bit = SBox(left(bit), S0) + SBox(right(bit), S1)
    bit = permutation(bit, P4)
    return xor(bit, L)

def encrypt(plain):
    bit = permutation(plain, IP) 
    tmp = Fk(bit, genfirstKey())
    bit = right(bit) + tmp
    bit = Fk(bit, gensecondKey())
    return permutation(bit + tmp, IP_INVERSE)

def decrypt(enc):
    bit = permutation(enc, IP)
    tmp = Fk(bit, gensecondKey())
    bit = right(bit) + tmp
    bit = Fk(bit, genfirstKey())
    return permutation(bit + tmp, IP_INVERSE)

# Main function
def main():
    _plain = input("Input the plain text: ")
    print('Plain text :', _plain)
    
    _encry = [encrypt(bin(ord(x))[2:].zfill(8)) for x in _plain]
    _decry = [chr(int(decrypt(x), 2)) for x in _encry]

    print('Encrypted  :', _encry)
    print('Decrypted  :', ''.join(_decry))

# Corrected main entry point
if __name__ == '__main__':
    main()


Input the plain text:  hello


Plain text : hello
Encrypted  : ['11101111', '01010100', '10101010', '10101010', '11000011']
Decrypted  : hello
